<a href="https://colab.research.google.com/github/JCaballerot/Recommender-Systems/blob/main/ALS_Recommender/ALS_Collaborative_Filtering_Last_fm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> ALS Collaborative Filtering - Last.fm </font></h1>

---

**Índice**

- 1. Introducción
- 2. Carga y Filtrado de Datos
- 3. Creación del Modelo ALS
- 4. Generación de Recomendaciones
- 5. Validación
- 6. Conclusiones


## 1. Introducción

Este laboratorio aplica Mínimos Cuadrados Ordinarios (OLS) con filtrado "long tail" a los datos de interacciones usuario-artista en Last.fm. Exploraremos cómo utilizar regresión lineal para predecir las interacciones entre usuarios y artistas. Evaluaremos el modelo ocultando el 20% de los ítems por usuario para probar la capacidad predictiva del modelo.

Instalamos las librerías necesarias.



In [ ]:
# Instalar librerías necesarias
!pip install scikit-surprise kaggle

# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Dataset, Reader, BaselineOnly
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from sklearn.model_selection import train_test_split as sk_train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


## 2. Carga y Filtrado de Datos

Cargamos el dataset y aplicamos un filtro "long tail" para mejorar la calidad del análisis, manteniendo solo los artistas con al menos 50 escuchas. Este enfoque reduce el impacto de artistas menos populares y permite centrarse en recomendaciones más relevantes.

In [ ]:
# Descargar el dataset de Last.fm desde Kaggle
!pip install kaggle

from google.colab import files
files.upload()  # Sube tu archivo kaggle.json aquí

# Configurar Kaggle API
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Descargar y descomprimir el dataset de Last.fm
!kaggle datasets download -d japarra27/lastfm-dataset
!unzip lastfm-dataset.zip


In [54]:
# Cargar el dataset
data = pd.read_parquet("lastfm_union.parquet")[:10_000_000]


In [ ]:
data.head()

**Filtrado "long tail"**


In [ ]:
# Contar las escuchas por artista
artist_listen_counts = data.groupby('artist_name').size().sort_values(ascending=False)

# Visualizar distribución long tail
plt.figure(figsize=(12, 6))
plt.bar(range(len(artist_listen_counts)), artist_listen_counts, color='lightblue')
plt.title('Distribución del Número de Escuchas por Artista (Long Tail)')
plt.xlabel('Artistas ordenados por popularidad')
plt.ylabel('Número de escuchas')
plt.ylim(1, 4000)
plt.show()


In [ ]:
artist_listen_counts

In [59]:
# Filtrar artistas con al menos 100 escuchas
min_listens_per_artist = 100
popular_artists = artist_listen_counts[artist_listen_counts >= min_listens_per_artist].index
data_filtered = data[data['artist_name'].isin(popular_artists)]


In [ ]:
# Filtrar usuarios con al menos 100 escuchas
users_listen_counts = data_filtered.groupby('user_id').size().sort_values(ascending=False)
users_listen_counts

In [61]:
min_listens_per_user = 100
popular_users = users_listen_counts[users_listen_counts >= min_listens_per_user].index
data_filtered = data_filtered[data_filtered['user_id'].isin(popular_users)]


In [ ]:
data_filtered.groupby('user_id').size().sort_values(ascending=False).tail()

In [ ]:
data_filtered.groupby('artist_name').size().sort_values(ascending=False).tail()

## 3. Creación del Modelo ALS

Ahora que tenemos los datos filtrados, procederemos a entrenar un modelo de recomendación usando ALS con la biblioteca surprise. En surprise, podemos utilizar el algoritmo BaselineOnly con el método de estimación configurado como ALS.

Primero, preparamos los datos en el formato que requiere surprise.

In [ ]:
# Crear el DataFrame con el recuento de escuchas
user_artist_df = data_filtered.groupby(['user_id', 'artist_name']).size().reset_index(name='listens')
user_artist_df

In [ ]:
np.percentile(user_artist_df['listens'], 95)

In [66]:
trainset_scaled = user_artist_df
trainset_scaled['listens'] = user_artist_df.listens/np.percentile(user_artist_df['listens'], 95)

**División del Conjunto de Datos**


In [ ]:
trainset_scaled

In [68]:
# Dividir en conjuntos de entrenamiento y prueba estratificando por usuario
train_data, test_data = sk_train_test_split(trainset_scaled,
    test_size = 0.2,
    random_state = 42,
    stratify = trainset_scaled['user_id']
)

## 4. Creación del Modelo ALS

Preparamos los datos para surprise y entrenamos el modelo ALS utilizando la clase BaselineOnly con el método als.

Preparación de los datos para Surprise

In [69]:
# Definir el rango de puntuaciones basado en los datos de entrenamiento
rating_scale = (train_data['listens'].min(), train_data['listens'].max())
reader = Reader(rating_scale=rating_scale)

# Crear el conjunto de datos de entrenamiento para 'surprise'
train_dataset = Dataset.load_from_df(
    train_data[['user_id', 'artist_name', 'listens']],
    reader
)
trainset = train_dataset.build_full_trainset()

# Crear el conjunto de prueba en formato de lista de tuplas
testset = list(
    test_data[['user_id', 'artist_name', 'listens']].itertuples(index=False, name=None)
)


Utilizaremos el modelo BaselineOnly de surprise con el método de estimación als para implementar ALS.

In [70]:
from surprise.model_selection import GridSearchCV

# Definir el espacio de búsqueda para los hiperparámetros
param_grid = {
    'bsl_options': {
        'method': ['als'],
        'n_epochs': [100],
        'reg_u': [0, 1, 5, 10, 15],
        'reg_i': [0, 1, 5, 10, 15]
    }
}

# Configurar el objeto GridSearchCV
gs = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=3)

# Ejecutar la búsqueda
gs.fit(train_dataset)

# Obtener los mejores hiperparámetros
best_params = gs.best_params['rmse']
print(f"Mejores hiperparámetros: {best_params}")


Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimati

In [71]:
# Configuración de los parámetros de ALS
bsl_options = {
    'method': 'als',
    'n_epochs': 1000,
    'reg_u': 15,
    'reg_i': 15
}

# Crear y entrenar el modelo ALS
als_model = BaselineOnly(bsl_options = bsl_options)
als_model.fit(trainset)


Estimating biases using als...


In [ ]:
from sklearn.metrics import r2_score

# Generar predicciones en el conjunto de entrenamiento
trainset_predictions = als_model.test(trainset.build_testset())

# Extraer las calificaciones reales y predichas
y_true_train = [pred.r_ui for pred in trainset_predictions]
y_pred_train = [pred.est for pred in trainset_predictions]

# Calcular el R² en el conjunto de entrenamiento
r2_train = r2_score(y_true_train, y_pred_train)
print(f"R² en el conjunto de entrenamiento: {r2_train}")


R² en el conjunto de entrenamiento: 0.08873116760890731


## 4. Generación de Recomendaciones


Utilizaremos el modelo entrenado para predecir las escuchas en el conjunto de prueba.



In [ ]:
# Realizar predicciones en el conjunto de prueba
predictions = als_model.test(testset)
